In [ ]:
import os
import numpy as np
import tensorflow as tf
from typing import Sequence, Optional
import math

In [ ]:
class input(object):
    def __init__(self, filePaths:Sequence[str], labels:Sequence[float], startTimes:Optional[Sequence[float]]=None, endTimes:Optional[Sequence[float]]=None, batchSize:Optional[int]=None):
        # Check that data has consistent shape
        cols = [filePaths, labels, startTimes, endTimes]
        if any(len(col)!=len(cols[0]) for col in cols if col is not None):
            raise Exception(f'Expected lists with equal length, but received lists with lengths {[len(col) for col in cols if col is not None]}')
        # Check that startTime and endTime are either both specified or both unspecified
        if all(type(startTimes)!=type(endTimes), type(startTimes) in (list, None))
        # Check that batch size is not larger than the total amount of data
        if batchSize is not None and batchSize > len(filePaths):
            raise Exception(f'batchSize={batchSize} cannot be larger than the length of filePaths')

        self.__filePaths = filePaths
        self.__labels = labels
        self.__startTimes = startTimes if startTimes is not None else [None]*len(filePaths)
        self.__endTimes = endTimes if endTimes is not None else [None]*len(filePaths)

        self.totalSize = len(filePaths)
        self.batchSize = batchSize
        self.numBatches = math.floor(self.totalSize / batchSize) if batchSize is not None else 0

        def dataset(self) -> tf.data.Dataset:
            try:
                out = self.__dataset
            except AttributeError:
                datasetCols = [self.filePaths, self.labels, self.startTimes, self.endTimes]
                self.__dataset = (
                    tf.data.Dataset.from_tensor_slices(datasetCols)
                    .map(self.__preprocessor, num_parallel_calls=tf.data.AUTOTUNE)
                )
            return out

        def __preprocessor(filePath, label, startTime, endTime):
            # Load and decode audio file
            encodedAudio = tf.io.read_file(filePath)
            rawAudio = tf.audio.decode_wav(encodedAudio)

            if startTime is not None and endTime is not None:
                
            pass